In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import nltk  
import re 
from string import punctuation
from nltk.tokenize import WordPunctTokenizer, PunktSentenceTokenizer
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

import string
import os
import pickle

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import Input, Embedding, LSTM, Dense

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\43664\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\43664\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\43664\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\43664\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ImportError: cannot import name 'pad_sequences' from 'keras.preprocessing.sequence' (C:\Users\43664\anaconda3\lib\site-packages\keras\preprocessing\sequence.py)

## Read and inspect data

In [3]:
raw_train_data = pd.read_csv('train.csv')
raw_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404290 non-null  int64 
 1   qid1          404290 non-null  int64 
 2   qid2          404290 non-null  int64 
 3   question1     404289 non-null  object
 4   question2     404288 non-null  object
 5   is_duplicate  404290 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [96]:
raw_train_data.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [97]:
raw_train_data.dtypes

id               int64
qid1             int64
qid2             int64
question1       object
question2       object
is_duplicate     int64
dtype: object

In [4]:
raw_test_data = pd.read_csv('test.csv')
raw_test_data.head(5)

C:\Users\43664\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [98]:
print(raw_train_data.isnull().sum())

id              0
qid1            0
qid2            0
question1       1
question2       2
is_duplicate    0
dtype: int64


In [5]:
raw_test_data.isnull().sum()

test_id      0
question1    4
question2    6
dtype: int64

## Data Preparation

In [8]:
train_data = raw_train_data.dropna(axis=0)
test_data = raw_train_data.dropna(axis=0)

In [100]:
train_data.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [9]:
test_data.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [12]:
train_data = train_data.drop(['id', 'qid1', 'qid2'], axis=1)
train_data.head(5)

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [104]:
train_data['question1'].iloc[0]

'What is the step by step guide to invest in share market in india?'

In [215]:
train_copy = train_data.copy()

## Text Preparation

In [216]:
## REMOVE ACCENTED CHARACTERS Sómě Áccěntěd těxt ##
import unicodedata

def remove_accented_chars_func(text):
    return unicodedata.normalize('NFKD', str(text)).encode('ascii', 'ignore').decode('utf-8', 'ignore')

## REMOVE PUNCTUATION ##

def remove_punctuation_func(text):
    return re.sub(r'[^a-zA-Z0-9]', ' ', str(text))

## REMOVE IRRELEVANT CHARACTERS 

def remove_irr_char_func(text):
    return re.sub(r'[^a-zA-Z]', ' ', str(text))

## REMOVE EXTRA WHITESPACEs ##

def remove_extra_whitespaces_func(text):
    return re.sub(r'^\s*|\s\s*', ' ', str(text)).strip()

## COUNT OF STRINGS ##

def word_count_func(text):
    return len(str(text).split())

In [217]:
train_copy['question1'] = train_copy['question1'].str.lower()
train_copy['question2'] = train_copy['question2'].str.lower()

train_copy['question1'] = train_copy['question1'].apply(remove_accented_chars_func)
train_copy['question2'] = train_copy['question2'].apply(remove_accented_chars_func)

train_copy['question1'] = train_copy['question1'].apply(remove_punctuation_func)
train_copy['question2'] = train_copy['question2'].apply(remove_punctuation_func)

train_copy['question1'] = train_copy['question1'].apply(remove_irr_char_func)
train_copy['question2'] = train_copy['question2'].apply(remove_irr_char_func)

train_copy['question1'] = train_copy['question1'].apply(remove_extra_whitespaces_func)
train_copy['question2'] = train_copy['question2'].apply(remove_extra_whitespaces_func)

train_copy.head(5)

,question1,question2,is_duplicate
0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0
1,what is the story of kohinoor koh i noor diamond,what would happen if the indian government sto...,0
2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0
3,why am i mentally very lonely how can i solve it,find the remainder when math math is divided by,0
4,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water,0


## Split data

In [218]:
features = ['question1', 'question2']

X_data = train_copy[features]
X_data.head()

,question1,question2
0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...
1,what is the story of kohinoor koh i noor diamond,what would happen if the indian government sto...
2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...
3,why am i mentally very lonely how can i solve it,find the remainder when math math is divided by
4,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water


In [219]:
y_data = train_copy['is_duplicate']
y_data.head(5)

0    0
1    0
2    0
3    0
4    0
Name: is_duplicate, dtype: int64

In [220]:
# Split the set into train and split data into 60/40, random_state 0

from sklearn.model_selection import train_test_split
X_train, X_split, y_train, y_split = train_test_split(X_data, y_data, test_size=0.90, random_state = 0)  ## cambiar test size

print("Whole dataset: ", len(train_copy))
print("X_train size =", len(X_train))
print("X_split size =", len(X_split))
print("y_train size =", len(y_train))
print("y_split size =", len(y_split))

Whole dataset:  404287
X_train size = 40428
X_split size = 363859
y_train size = 40428
y_split size = 363859


In [139]:
# Split the set into test and validation into 50/50 (from 40% of the splitted data), random_state 0

from sklearn.model_selection import train_test_split
X_test, X_val, y_test, y_val = train_test_split(X_split, y_split, test_size=0.50, random_state = 0)

print("Whole dataset: ", len(train_copy))
print("X_test size =", len(X_test))
print("X_val size =", len(X_val))
print("y_test size =", len(y_test))
print("y_val size =", len(y_val))

Whole dataset:  404287
X_test size = 80857
X_val size = 80858
y_test size = 80857
y_val size = 80858


In [143]:
X_val.head(5)

,question1,question2
189347,what are the most asked mainframe interview qu...,what is the most asked question in an interview
327140,how long should i wait for my bitcoin core wal...,can one set up a bitcoin wallet under false id...
126940,i got selected for internship at inria th june...,i got selected for internship at inria th june...
78406,how do you stay fit if you don t like the gym,how can i stay fit with out going gym
372855,what is the best budget hotels in cochin for a...,what is the best budget hotels in mussoorie fo...


## Text Normalization

## Tokenization

In [221]:
### X_train ###
## QUESTION 1 ##
X_train_text_q1 = X_train['question1'] 

X_train_q1 = []

for sent_train_q1 in X_train_text_q1:
    
    #lowercase
    #sent = sent.lower()
  
    sent_train_q1 = sent_train_q1.translate(str.maketrans('', '', string.punctuation))

  ## tokenize ##
    train_tokens_q1 = nltk.word_tokenize(sent_train_q1)

  ## remove stop-words ##
    english_stopwords = stopwords.words('english')
    #english_stopwords = english_stopwords + ['did', 'could', 'would', 'have', 'had', 'has', 'might', 'should', 'was', 'were', 'does', 'much', 'br', 'n']
    train_tokens_q1 = [el for el in train_tokens_q1 if not el in english_stopwords]

  ## lemmatization ##
 
    #train_tokens_lemma_q1 = [wnl.lemmatize(el, get_wordnet_pos(nltk.pos_tag([el])[0][1])) for el in train_tokens_q1]

  ## add to output list ##
    X_train_q1.append(" ".join(train_tokens_q1))

In [222]:
### X_train ###
## QUESTION 2 ##
X_train_text_q2 = X_train['question2'] 

X_train_q2 = []

for sent_train_q2 in X_train_text_q2:
    
    #lowercase
    #sent = sent.lower()
  
    sent_train_q2 = sent_train_q2.translate(str.maketrans('', '', string.punctuation))

  ## tokenize ##
    train_tokens_q2 = nltk.word_tokenize(sent_train_q2)

  ## remove stop-words ##
    english_stopwords = stopwords.words('english')
    #english_stopwords = english_stopwords + ['did', 'could', 'would', 'have', 'had', 'has', 'might', 'should', 'was', 'were', 'does', 'much', 'br', 'n']
    train_tokens_q2 = [el for el in train_tokens_q2 if not el in english_stopwords]

  ## lemmatization ##
 
    #train_tokens_lemma_q2 = [wnl.lemmatize(el, get_wordnet_pos(nltk.pos_tag([el])[0][1])) for el in train_tokens_q1]

  ## add to output list ##
    X_train_q2.append(" ".join(train_tokens_q2))

In [153]:
### X_test ###
## QUESTION 1 ##
X_test_text_q1 = X_test['question1'] 

X_test_q1 = []

for sent_test_q1 in X_test_text_q1:
    
    #lowercase
    #sent = sent.lower()
  
    sent_test_q1 = sent_test_q1.translate(str.maketrans('', '', string.punctuation))

  ## tokenize ##
    test_tokens_q1 = nltk.word_tokenize(sent_test_q1)

  ## remove stop-words ##
    english_stopwords = stopwords.words('english')
    #english_stopwords = english_stopwords + ['did', 'could', 'would', 'have', 'had', 'has', 'might', 'should', 'was', 'were', 'does', 'much', 'br', 'n']
    test_tokens_q1 = [el for el in test_tokens_q1 if not el in english_stopwords]

  ## lemmatization ##
 
    #test_tokens_lemma_q1 = [wnl.lemmatize(el, get_wordnet_pos(nltk.pos_tag([el])[0][1])) for el in test_tokens_q1]

  ## add to output list ##
    X_test_q1.append(" ".join(test_tokens_q1))

In [154]:
### X_test ###
## QUESTION 1 ##
X_test_text_q2 = X_test['question2'] 

X_test_q2 = []

for sent_test_q2 in X_test_text_q2:
    
    #lowercase
    #sent = sent.lower()
  
    sent_test_q2 = sent_test_q2.translate(str.maketrans('', '', string.punctuation))

  ## tokenize ##
    test_tokens_q2 = nltk.word_tokenize(sent_test_q2)

  ## remove stop-words ##
    english_stopwords = stopwords.words('english')
    #english_stopwords = english_stopwords + ['did', 'could', 'would', 'have', 'had', 'has', 'might', 'should', 'was', 'were', 'does', 'much', 'br', 'n']
    test_tokens_q2 = [el for el in test_tokens_q2 if not el in english_stopwords]

  ## lemmatization ##
 
    #test_tokens_lemma_q2 = [wnl.lemmatize(el, get_wordnet_pos(nltk.pos_tag([el])[0][1])) for el in test_tokens_q2]

  ## add to output list ##
    X_test_q2.append(" ".join(test_tokens_q2))

In [160]:
### X_val ###
## QUESTION 1 ##
X_val_text_q1 = X_val['question1'] 

X_val_q1 = []

for sent_val_q1 in X_val_text_q1:
    
    #lowercase
    #sent = sent.lower()
  
    sent_val_q1 = sent_val_q1.translate(str.maketrans('', '', string.punctuation))

  ## tokenize ##
    val_tokens_q1 = nltk.word_tokenize(sent_val_q1)

  ## remove stop-words ##
    english_stopwords = stopwords.words('english')
    #english_stopwords = english_stopwords + ['did', 'could', 'would', 'have', 'had', 'has', 'might', 'should', 'was', 'were', 'does', 'much', 'br', 'n']
    val_tokens_q1 = [el for el in val_tokens_q1 if not el in english_stopwords]

  ## lemmatization ##
 
    #val_tokens_lemma_q1 = [wnl.lemmatize(el, get_wordnet_pos(nltk.pos_tag([el])[0][1])) for el in val_tokens_q1]

  ## add to output list ##
    X_val_q1.append(" ".join(val_tokens_q1))

In [162]:
### X_val ###
## QUESTION 2 ##
X_val_text_q2 = X_val['question2'] 

X_val_q2 = []

for sent_val_q2 in X_val_text_q2:
    
    #lowercase
    #sent = sent.lower()
  
    sent_val_q2 = sent_val_q2.translate(str.maketrans('', '', string.punctuation))

  ## tokenize ##
    val_tokens_q2 = nltk.word_tokenize(sent_val_q2)

  ## remove stop-words ##
    english_stopwords = stopwords.words('english')
    #english_stopwords = english_stopwords + ['did', 'could', 'would', 'have', 'had', 'has', 'might', 'should', 'was', 'were', 'does', 'much', 'br', 'n']
    val_tokens_q2 = [el for el in val_tokens_q2 if not el in english_stopwords]

  ## lemmatization ##
 
    #val_tokens_lemma_q2 = [wnl.lemmatize(el, get_wordnet_pos(nltk.pos_tag([el])[0][1])) for el in val_tokens_q2]

  ## add to output list ##
    X_val_q2.append(" ".join(val_tokens_q2))

In [21]:
train_copy['question1'][:10]

0    What is the step by step guide to invest in sh...
1    What is the story of Kohinoor (Koh-i-Noor) Dia...
2    How can I increase the speed of my internet co...
3    Why am I mentally very lonely? How can I solve...
4    Which one dissolve in water quikly sugar, salt...
5    Astrology: I am a Capricorn Sun Cap moon and c...
6                                  Should I buy tiago?
7                       How can I be a good geologist?
8                      When do you use シ instead of し?
9    Motorola (company): Can I hack my Charter Moto...
Name: question1, dtype: object

In [163]:
X_val_q2[:10] 

['asked question interview',
 'one set bitcoin wallet false identity',
 'got selected internship inria th june th sep eligible charpak research scholarship please specify long',
 'stay fit going gym',
 'best budget hotels mussoorie accommodation friends coming new year eve',
 'property price decrease india money demonetisation',
 'color mn',
 'donald j trump presidency affect opportunities offered non us students',
 'best coaching institute gate chandigarh',
 'benjamin netanyahu display childish behaviors consistent basis']

In [70]:
# df_q1 = pd.DataFrame(train_text_q1, columns=['question1'])
# df_q2 = pd.DataFrame(train_text_q2, columns=['question2'])
# #train_df = pd.concat([df_q1, df_q2], axis=0)
# train_df = df_q1.join(df_q2, lsuffix='_left', rsuffix='_right')
# print(train_df.head(5))

                                           question1  \
0          step step guide invest share market india   
1                    story kohinoor kohinoor diamond   
2       increase speed internet connection using vpn   
3                              mentally lonely solve   
4  one dissolve water quikly sugar salt methane c...   

                                           question2  
0                step step guide invest share market  
1  would happen indian government stole kohinoor ...  
2               internet speed increased hacking dns  
3           find remainder math2324math divided 2423  
4                      fish would survive salt water  
                                           question1
0          step step guide invest share market india
1                    story kohinoor kohinoor diamond
2       increase speed internet connection using vpn
3                              mentally lonely solve
4  one dissolve water quikly sugar salt methane c...
               

In [208]:
df_q1 = pd.DataFrame(X_train_q1, columns=['question1'])
df_q2 = pd.DataFrame(X_train_q2, columns=['question2'])
# #train_df = pd.concat([df_q1, df_q2], axis=0)
train_df = df_q1.join(df_q2, lsuffix='_left', rsuffix='_right')
train_df

,question1,question2
0,best pick line person ever said,epic pick line ever
1,meaning cc engine,cc engine indicate
2,borivali churchgate kalyan cst journey better,someone think think soulmate
3,cool psychological hacks,reverse psychology life hacks
4,create empire st century,ever empire st century
...,...,...
242567,java android,r java android
242568,proper length men pants,proper length men suit pants
242569,know whether want mbbs something else,know really want become doctor
242570,hack snapchat chat history,spy snapchat chat history


In [207]:
train_df

,question1,question2
0,best pick line person ever said,epic pick line ever
1,meaning cc engine,cc engine indicate
2,borivali churchgate kalyan cst journey better,someone think think soulmate
3,cool psychological hacks,reverse psychology life hacks
4,create empire st century,ever empire st century
...,...,...
242567,java android,r java android
242568,proper length men pants,proper length men suit pants
242569,know whether want mbbs something else,know really want become doctor
242570,hack snapchat chat history,spy snapchat chat history


# Using Spacy 

In [169]:
def spacy_process(text):
    doc = nlp(text)
    
    #Tokenization and lemmatization are done with the spacy nlp pipeline commands
    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)
    print("Tokenize+Lemmatize:")
    print(lemma_list)
    
    #Filter the stopword
    filtered_sentence =[] 
    for word in lemma_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
    
    #Remove punctuation
    punctuations="?:!.,;"
    for word in filtered_sentence:
        if word in punctuations:
            filtered_sentence.remove(word)
    print(" ")
    print("Remove stopword & punctuation: ")
    print(filtered_sentence)

In [182]:
nlp = spacy.load('en_core_web_sm')

In [185]:
#!python -m spacy download en_core_web_sm

In [186]:
#import spacy
spacy_X_train_q1 = spacy_process(str(X_train['question1']))

Tokenize+Lemmatize:
['15296', '    ', 'what', 'be', 'the', 'good', 'pick', 'up', 'line', 'a', 'person', 'have', 'eve', '...', '\n', '223726', '                 ', 'what', 'be', 'the', 'meaning', 'of', 'cc', 'in', 'engine', '\n', '276564', '   ', 'borivali', 'to', 'churchgate', 'or', 'kalyan', 'to', 'cst', 'which', '...', '\n', '171976', '              ', 'what', 'be', 'some', 'cool', 'psychological', 'hack', '\n', '107068', '       ', 'how', 'do', 'you', 'create', 'an', 'empire', 'in', 'the', 'st', 'century', '\n                                ', '...', '                       \n', '359785', '                           ', 'what', 'be', 'java', 'and', 'android', '\n', '358085', '           ', 'what', 'be', 'the', 'proper', 'length', 'for', 'man', 's', 'pant', '\n', '152316', '   ', 'how', 'do', 'I', 'know', 'whether', 'I', 'want', 'to', 'do', 'mbb', 'or', 'som', '...', '\n', '117953', '            ', 'how', 'can', 'I', 'hack', 'the', 'snapchat', 'chat', 'history', '\n', '305713', '     

In [190]:
print(spacy_X_train_q1)

None


In [191]:
import nltk
from nltk.tokenize import word_tokenize  
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *

p_stemmer = PorterStemmer()
#s_stemmer = SnowballStemmer(language='english')

def nltk_process(text):
    #Tokenization
    nltk_tokenList = word_tokenize(text)
    
    #Stemming
    nltk_stemedList = []
    for word in nltk_tokenList:
        nltk_stemedList.append(p_stemmer.stem(word))
        #nltk_stemedList.append(s_stemmer.stem(word))
    
    #Lemmatization
    wordnet_lemmatizer = WordNetLemmatizer()
    nltk_lemmaList = []
    for word in nltk_stemedList:
        nltk_lemmaList.append(wordnet_lemmatizer.lemmatize(word))
    
    print("Stemming + Lemmatization")
    print(nltk_lemmaList)

    #Filter stopword
    filtered_sentence = []  
    nltk_stop_words = set(stopwords.words("english"))
    for w in nltk_lemmaList:  
        if w not in nltk_stop_words:  
            filtered_sentence.append(w)  

    #Removing Punctuation
    punctuations="?:!.,;"
    for word in filtered_sentence:
        if word in punctuations:
            filtered_sentence.remove(word)
    print(" ")
    print("Remove stopword & Punctuation")
    print(filtered_sentence)

In [193]:
nltk_process(str(X_train_q2))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

## Labelling target variable

In [164]:
labels = preprocessing.LabelEncoder()
# Convert continous y values to categorical
y_train_cat= labels.fit_transform(y_train)
print(y_train_cat)

y_val_cat = labels.fit_transform(y_val)
print(y_val_cat)

y_test_cat = labels.fit_transform(y_test)
print(y_test_cat)

[0 1 0 ... 1 1 1]
[0 0 0 ... 0 0 1]
[0 0 0 ... 0 0 1]


## Models 


In [88]:
X_val

,question1,question2
187265,differences german austrian culture,cultural differences austria germany
6557,best doctor doom stories,best comics featuring doctor doom
139264,best books prepare ssb,best books prepare iit jam physics exam
240135,ms vlsi portland state university good heard i...,universities apply ms vlsi
3336,height would 14 year old increase height,increase height
...,...,...
33649,recommendation cheap hosting plan host 3 diffe...,host go daddy registered domain wordpress
120451,mean blood bright red,blood dark red
256699,trump want lose election possible evidence stu...,trump really want president selfdestructive stuff
64328,lewis structure c4h8 determined,lewis structure n2f4 determined


In [205]:
X_train_q1

['best pick line person ever said',
 'meaning cc engine',
 'borivali churchgate kalyan cst journey better',
 'cool psychological hacks',
 'create empire st century',
 'post question quora posted many rejected',
 'partner get stop cuming quickly',
 'best military instructional videos',
 'wrong stephen colbert right ear',
 'prisoners often think jail',
 'prepare clat',
 'fastest way charge iphone',
 'overcome despair depression',
 'best political leaders india',
 'best golf wedge money',
 'best book understand theory relativity',
 'side effects protienx',
 'best pomodoro app windows',
 'solo traveler get laid',
 'get complete list gmail accounts',
 'valid reason dumped making partner wait sexual intimacy',
 'characteristics ron harry hermione bring table',
 'could happen lie bed one week smartphone food water',
 'donald trump become racist',
 'possible guy find guys attractive gay',
 'effect braces hearing',
 'long take see results intermittent fasting',
 'difference procedural programmi

In [223]:

from sklearn.feature_extraction.text import CountVectorizer

questions = list(X_train_q1) + list(X_train_q2)

count_vect = CountVectorizer()
corpus =  [questions]

X_train_q1_arr, X_train_q2_arr = np.vsplit(count_vect.fit_transform(questions).toarray(), 2)

#corpus = [train_text_q1, train_text_q2]

X_train_counts = count_vect.fit_transform(X_train_q1_arr, X_train_q2_arr)


MemoryError: Unable to allocate 17.2 GiB for an array with shape (80856, 28594) and data type int64

In [213]:
from sklearn.feature_extraction.text import CountVectorizer

questions = list(train_limited['question1']) + list(train_limited['question2'])

count_vect = CountVectorizer()
corpus =  [questions]

q1_arr, q2_arr = np.vsplit(count_vect.fit_transform(questions).toarray(), 2)

#corpus = [train_text_q1, train_text_q2]

X_train_counts = count_vect.fit_transform(q1_arr, q2_arr)

#pd.DataFrame(X_train_counts.toarray(),columns=count_vect.get_feature_names(),index=['question1','question2'])


AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [212]:
train_limited = train_df[:10]
train_limited

,question1,question2
0,best pick line person ever said,epic pick line ever
1,meaning cc engine,cc engine indicate
2,borivali churchgate kalyan cst journey better,someone think think soulmate
3,cool psychological hacks,reverse psychology life hacks
4,create empire st century,ever empire st century
5,post question quora posted many rejected,best ways ask question quora
6,partner get stop cuming quickly,premature ejaculation
7,best military instructional videos,right say difference tangential velocity orbit...
8,wrong stephen colbert right ear,thing inside stephen colbert desk
9,prisoners often think jail,often people escape maximum security prisons


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

trsfm=vectorizer.fit_transform(corpus)



from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(trsfm[0:1], trsfm)

cosine_similarity(trsfm[0:1], trsfm)

In [96]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(#texto 1 , texto 2)

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_2216/819094467.py, line 3)